In [12]:
%load_ext autoreload
%autoreload 2
import os
from langchain.chat_models import AzureChatOpenAI
import time
from dotenv import load_dotenv
from datetime import datetime
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from dateutil import parser
import time
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Load environment file
load_dotenv('.env')

True

In [14]:
# IMPORTANT: MongoDB setup
uri = "mongodb+srv://yh:yh@clinicalnotesreviewer.27ne3.mongodb.net/"
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['ClinicalNotesReviewer']
collection = db['processed_reports']
comparison_collection = db['comparison_gpt_table_test']

In [16]:
# Set chat variables
CHAT_API_VERSION = os.environ.get("OPENAI_CHAT_API_VERSION", "2023-05-15")
CHAT_DEPLOYMENT = os.environ.get("OPENAI_CHAT_DEPLOYMENT", "gpt-35-turbo")
CHAT_TEMPERATURE = float(os.environ.get("OPENAI_CHAT_TEMPERATURE", "0.0"))
CHAT_RESPONSE_MAX_TOKENS = int(os.environ.get(
    "OPENAI_CHAT_RESPONSE_MAX_TOKENS", "100"))

# openai keys
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]

# Set search variables
AZURE_SEARCH_SERVICE_ENDPOINT = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
SEARCH_MAX_RESULTS = int(os.environ.get("SEARCH_MAX_RESULTS", "3")) # If no value in environment, set it to 3


In [17]:
# memory for chat history, use the completion model to summarize past conversations: gpt3.5
# this is the model to generate search query terms
llm = AzureChatOpenAI(
    default_headers={"Ocp-Apim-Subscription-Key": os.environ["OPENAI_API_KEY"]}, # latest version of langchain
    # headers={"Ocp-Apim-Subscription-Key": os.environ["OPENAI_API_KEY"]},
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=CHAT_API_VERSION,
    # max_tokens=CHAT_RESPONSE_MAX_TOKENS,
    temperature=CHAT_TEMPERATURE,
    verbose=True
)

# this is the gpt 4o model
chat = AzureChatOpenAI(
    default_headers={"Ocp-Apim-Subscription-Key": os.environ["OPENAI_API_KEY"]}, # latest version of langchain
    # headers={"Ocp-Apim-Subscription-Key": os.environ["OPENAI_API_KEY"]},
    openai_api_base=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
    deployment_name="genai-GPT4o",
    model_name="gpt-4o",

    openai_api_version=CHAT_API_VERSION,
    # max_tokens=CHAT_RESPONSE_MAX_TOKENS,
    temperature=CHAT_TEMPERATURE,
    verbose=True
)

print(chat)

client=<openai.resources.chat.completions.Completions object at 0x00000160FE8B8C40> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000160FE8C8D60> model_name='gpt-4o' temperature=0.0 model_kwargs={} openai_api_key='ae9587f69088409992009cb7bcf61436' openai_api_base='https://genai-openai-eus.openai.azure.com/openai/deployments/genai-GPT4o' openai_proxy='' default_headers={'Ocp-Apim-Subscription-Key': 'ae9587f69088409992009cb7bcf61436'} openai_api_version='2023-05-15' openai_api_type='azure'


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://genai-openai-eus.openai.azure.com/ to https://genai-openai-eus.openai.azure.com/openai.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\chat_models\azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_community\chat_models\azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) 

In [18]:
# Adjust the date format in get_reports_by_patient function
def get_reports_by_patient():
    reports_by_patient = {}
    reports = collection.find({})

    for report in reports:
        try:
            # Parse using explicit date format to prevent issues
            performed_date_time = datetime.strptime(report['Performed Date Time'], "%d/%m/%Y %H:%M")
            patient_id = report['PatientID']

            if patient_id not in reports_by_patient:
                reports_by_patient[patient_id] = []
            reports_by_patient[patient_id].append((performed_date_time, report))
        except ValueError as e:
            print(f"Error parsing date for report {report['_id']}: {e}")
            continue

    return reports_by_patient

# Function to format radiology report
def format_radiology_report(report):
    formatted_report = (
        f"Patient ID: {report['Raw Report']['Masked_PatientID']}, Performed Date: {report['Performed Date Time']}\n\n"
        f"Raw Radiology Report Extracted\n"
        f"Text: {report['Raw Report']['Text'].strip()}\n\n"
    )

    processed_data = {
        "Diseases Mentioned": report['Processed Data']['Summary'].get('Diseases Mentioned', ""),
        "Organs Mentioned": report['Processed Data']['Summary'].get('Organs Mentioned', ""),
        "Symptoms/Phenomena of Concern": report['Processed Data']['Summary'].get('Symptoms/Phenomena of Concern', "")
    }

    return formatted_report, processed_data

In [19]:
template_prompt = """
    You are comparing two radiology reports in the section '{section_name}', where the content is provided as key-value pairs.\n\n
    
    ### Structure of Input:\n
    The input consists of observations represented as key-value pairs:\n
    In this comparison, the input for the Newer Report ({date1_str}) is: {section_content_1}\n
    and the input for the Older Report ({date2_str}) is: {section_content_2}\n\n

    ### Special Cases:\n
    1. **Both Reports Are Empty:**\n
       - If there are no key-value pairs in both the Newer Report and Older Report, output *Both report sections are empty*.\n\n
    2. **Newer Report is Empty, Older Report Has Key-Value Pairs:**\n
       - If the Newer Report contains no key-value pairs, but the Older Report does:\n
         - Categorize each key-value pair in the Older Report as **No Longer Mentioned**.\n
         - Use 'NIL' as the content for '{date1_str} Content'.\n
         - Use the values from the Older Report for '{date2_str} Content'.\n\n

    ### Comparison Instructions:\n
    Follow the step-by-step logic to compare the key-value pairs from the two reports:\n\n
    1. **Comparison Flow:**\n
       - Compare each key in the Newer Report ({date1_str}) against all keys in the Older Report ({date2_str}).\n
       - Treat keys that are phrased **similarly** or refer to the same concept as being the **same key**.\n\n

    2. **Categorization Logic:**\n
       - **CASE 1: Difference**\n
         - If a key in the Newer Report matches a key in the Older Report (or is phrased similarly):\n
           - Use the value from the Newer Report as '{date1_str} Content'.\n
           - Use the value from the Older Report as '{date2_str} Content'.\n
           - If the value from the Newer Report is the same or similar to the value of the Older Report (e.g., both values are 'Enlarged'): Categorized as **Difference**.\n\n
           - Else, DO NOT categorise as **Difference** and move on to the next key-value pair in Newer Report.\n\n
       - **CASE 2: New Development**\n
         - If the key in the Newer Report is **not present** in the Older Report (and no similar key exists):\n
           - Use the value from the Newer Report as '{date1_str} Content'.\n
           - Use 'NIL' for '{date2_str} Content'.\n
           - Categorize as **New Development**.\n\n
       - **CASE 3: No Longer Mentioned**\n
         - After processing all keys from the Newer Report, check the Older Report for any unused key-value pairs.\n
           - For each unused key-value pair:\n
             - Use 'NIL' for '{date1_str} Content'.\n
             - Use the value from the Older Report as '{date2_str} Content'.\n
             - Categorize as **No Longer Mentioned**.\n\n

    3. **Key Similarity Clarification:**\n
       - Keys that are similar in meaning or phrasing (e.g., 'Minor atelectasis' and 'Atelectasis (lung collapse)') must be treated as the **same key**.\n
       - These comparisons must always be categorized as **Difference** ONLY IF their values differ (e.g., both values are 'Enlarged').\n\n

    4. **Output Format:**\n
    Do not output irrelevant text like: Here's the comparison of the two radiology reports following the provided logic, output only the table.
    f
    Present the comparison results in the following table format:\n"
    | Category            | {date1_str} Content                      | {date2_str} Content                      | Explanation                         |\n
    |---------------------|------------------------------------------|------------------------------------------|-------------------------------------|\n
    | Difference          | {{Newer Report Value}}                   | {{Older Report Value}}                   | Explanation of the difference.      |\n
    | New Development     | {{Newer Report Value}}                   | NIL                                      | Explanation of new observation.     |\n
    | No Longer Mentioned | NIL                                      | {{Older Report Value}}                   | Explanation of removal or absence.  |\n\n
    
    ### Important Rules:\n
    1. Do **not** interpret or infer any information that is not explicitly stated in the provided key-value pairs.\n
    2. Treat similar keys as the **same key** and strictly follow the comparison flow and categorization rules.\n
    3. Use the exact wording from the reports for {date1_str} Content and {date2_str} Content.\n
    4. Make sure there is no duplication of entries across categories.\n\n
    Please proceed with the comparison following the above logic strictly.
"""

In [20]:
def compare_section(section_name, content1, content2, date1, date2):

    date1_str = date1.strftime("%d/%m/%Y %H:%M:%S")
    date2_str = date2.strftime("%d/%m/%Y %H:%M:%S")

    prompt_t = PromptTemplate.from_template(template_prompt)

    # Create an LLMChain using the PromptTemplate
    gpt_chain = LLMChain(llm=chat, prompt=prompt_t)

    retries = 3  # Number of retries for the API call
    for attempt in range(retries):
        try:
            # Send the prompt to the model via the chain
            response = gpt_chain.invoke({
                'section_name': section_name,
                'section_content_1': content1,
                'section_content_2': content2,
                'date1_str': date1_str,
                'date2_str': date2_str
            })

            # debug prints
            print(f"raw response: {response}")
            comparison_content = response.get('text', '')
            print(f"extracted response: {comparison_content}")


            # Ensure response is in text form (if response is directly a string, you don't need .text)
            if isinstance(comparison_content, str):
                comparison_output = comparison_content.strip()
            else:
                comparison_output = ""

            if comparison_output:
                # Parse the comparison output into a structured format
                def parse_comparison_headers(comparison_string):
                    lines = comparison_string.strip().split("\n")
                    headers = [header.strip() for header in lines[0].split("|")[1:-1]]
                    comparison_list = []
                    
                    for line in lines[2:]:  # Skip the header line and separator
                        parts = line.split("|")
                        # Extract the key-value pairs into a structured dictionary
                        comparison_entry = {headers[i]: parts[i+1].strip() for i in range(len(headers))}
                        comparison_list.append(comparison_entry)
                    
                    print(f"comparison_list: {comparison_list}")
                    return comparison_list

                # Parse the structured comparison from the output
                structured_comparison = parse_comparison_headers(comparison_output)
                return structured_comparison
            else:
                print(f"No valid comparison output for section '{section_name}'")
                return []

        except Exception as e:
            if "429" in str(e):
                print(f"API quota exceeded. Retrying in 30 seconds... (Attempt {attempt + 1}/{retries})")
                time.sleep(30)
            else:
                print(f"Error generating comparison for section '{section_name}': {e}")
                return []

    print(f"Max retries reached. Could not generate comparison for section '{section_name}'.")
    return []


# Function to compare multiple reports
def compare_multiple_reports(reports):
    reports.sort(key=lambda x: x[0], reverse=True)
    base_report = reports[0]
    all_comparisons = []

    for i in range(1, len(reports)):
        report = reports[i]
        base_text, base_sections = format_radiology_report(base_report[1])
        report_text, report_sections = format_radiology_report(report[1])

        # loop through comparison prompt for each section
        for section_name in base_sections.keys():

            # Debug prints
            content1 = base_sections[section_name]
            content2 = report_sections[section_name]

            print(f"type of content 1: {type(content1)}")
            print(f"type of content 2: {type(content2)}")

            print(f"content 1 ({section_name}): {content1}")
            print(f"content 2 ({section_name}): {content2}")

            if not content1 and not content2:
                print(f"Both contents are empty for section '{section_name}'. Skipping comparison.")
                continue

            comparison_result = compare_section(
                section_name,
                content1,
                content2,
                base_report[0],
                report[0],
                
            )
            # Add the Section attribute to each comparison result and ensure it appears first
            for comparison in comparison_result:
                comparison = {
                    "Section": section_name, 
                    **comparison,
                    'New Report Date': base_report[0],
                    'Old Report Date': report[0],
                    'New Report Order ID': base_report[1]['Raw Report']['Order ID'],  
                    'Old Report Order ID': report[1]['Raw Report']['Order ID'],       
                    'New Report Order Name': base_report[1]['Raw Report']['Order Name'],
                    'Old Report Order Name': report[1]['Raw Report']['Order Name'], }
                
                all_comparisons.append(comparison)

    return all_comparisons

# Save comparisons to MongoDB
def save_comparisons(patient_id, report_dates, comparisons):
    output = {
        "PatientID": patient_id,
        "ReportDates": [date.strftime("%Y-%m-%d %H:%M:%S") for date in report_dates],
        "Comparisons": comparisons
    }
    comparison_collection.update_one(
        {"PatientID": patient_id},
        {"$set": output},
        upsert=True
    )
    print(f"Saved comparisons for PatientID {patient_id}.")


In [21]:
def main():
    reports_by_patient = get_reports_by_patient()
    for patient_id, reports in reports_by_patient.items():
        # Sort reports by performed date time in ascending order
        reports.sort(key=lambda x: x[0])  # Sort by datetime

        # Consider cases whereby there is only 1 report for the patient
        if len(reports) == 1:
            print(f"Only one report available for PatientID {patient_id}. No comparison will be generated.")

            # Create a simple JSON output for patients with only one report
            single_report_output = {
                "PatientID": patient_id,
                "ComparisonResults": "No comparison available as there is only one report for this patient."
            }

            # Save to MongoDB
            comparison_collection.update_one(
                {"PatientID": patient_id},  # Filter by PatientID
                {"$set": single_report_output},  # Set the new JSON output
                upsert=True  # Insert if no existing document is found
            )
            continue

        # If there are more than 5 reports, only keep the latest 5
        if len(reports) > 5:
            reports = reports[-5:]

        # Get the latest report date in the current reports list
        latest_report_date = reports[-1][0]

        # Check if there's already a comparison output for this patient
        existing_comparison = comparison_collection.find_one({"PatientID": patient_id})

        if existing_comparison:
            # Convert dates from existing comparison to datetime objects for comparison
            existing_dates = [parser.parse(date_str) for date_str in existing_comparison.get("ReportDates", [])]

            # Find the latest date in the existing comparison output
            latest_existing_date = max(existing_dates) if existing_dates else None

            # Check if there's a new report by comparing dates
            if latest_existing_date and latest_existing_date >= latest_report_date:
                print(f"No new reports for PatientID {patient_id}. Skipping comparison.")
                continue

        # Perform comparison since either there’s no existing comparison or new reports are present
        comparison_results = compare_multiple_reports(reports)

        print("after compare multiple reports")

        # Collect report dates for saving in the JSON output
        report_dates = [report[0] for report in reports]

        # Save comparison output to MongoDB
        save_comparisons(patient_id, report_dates, comparison_results)

if __name__ == "__main__":
    main()

after reports by patient
{'Patient1': [(datetime.datetime(2023, 2, 9, 15, 50), {'_id': ObjectId('676143d0562e23f5d28cedfa'), 'PatientID': 'Patient1', 'Performed Date Time': '9/2/2023 15:50', 'Raw Report': {'Visit Number': '3930f4bbc4f8ceaf060222ec14cec3d8cf99bdab7293240b00d973d2c13cf571', 'Masked_PatientID': '1', 'Order ID': 'b2d2dc2b49f2f0ed11c100718f0f9c445b88d1eafb302ace311c7df2e7d44685', 'Order Name': 'Chest X-ray', 'Result Item Code': 'CHE-NOV', 'Performed Date Time': '9/2/2023 15:50', 'Line Num': '1', 'Text': '      HISTORY post op REPORT  The right central venous line, endotracheal tube, feeding tube, left intercostal  drain tube are in situ and projected in satisfactory position.   The median sternotomy wires and vascular clips are intact The heart size cannot be accurately assessed in this projection but appears prominent.   Small left pleural effusion noted associated with possible underlying subsegmental  atelectasis and consolidation.  The right lung is unremarkable.  No ri